### Описание проекта

Задача обучить модель классифицировать комментарии. Нам доступен датасет с размеченными позитивными и негативными комментариями.

Целевая метрика качества: *F1* не меньше 0.75. 

### Описание данных

`text` - текст комментария  
`toxic` - целевой признак

## Содержание

1. [Подготовка](#1.-Подготовка)
2. [Обучение](#2.-Обучение)
3. [Вывод](3.-Вывод)

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
import torch
from tqdm import notebook
import transformers as ppb

In [2]:
#загружаем данные
df_toxic = pd.read_csv('./toxic_comments.csv')
df_toxic.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
#посмотрим как распределена целевая переменная
df_toxic['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [4]:
#посмотрим долю токсичных примеров
df_toxic['toxic'].mean()

0.10167887648758234

Выборка не сбалансирована. Это следует учесть при построении модели. 

In [5]:
def clear_text(text):
    #функция для очистки текста
    text = re.sub(r'[^a-zA-z]', ' ', text)
    text = text.split()
    text = ' '.join(text)
    return text

#очищаем текст
df_toxic['text'] = df_toxic['text'].apply(clear_text)
df_toxic.head()

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D aww He matches this background colour I m se...,0
2,Hey man I m really not trying to edit war It s...,0
3,More I can t make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0


In [6]:
def stemming(text):
    #функция для стемминга текста
    stemmer = SnowballStemmer('english')
    text = stemmer.stem(text)
    return text

#делаем стемминг текста
df_toxic['text'] = df_toxic['text'].apply(stemming)
df_toxic.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [7]:
#посмотрим общие данные
df_toxic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Данные почистили. Получилось 159 тыс. примеров, из них 10 % токсичные комментарии.

# 2. Обучение

### TF-IDF + Logreg

Векторизируем текст с помощью TF-IDF. На полученной матрице признаков обучим логистическую регрессию. 

In [8]:
#создаем копус
corpus = df_toxic['text'].values.astype('U')

In [9]:
#разделим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(corpus, df_toxic['toxic'], test_size=0.3)

del corpus

#проверяем размерность
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((111699,), (47872,), (111699,), (47872,))

In [10]:
#загружаем и сохраняем множество стоп-слов
nltk.download('stopwords')
stop_words = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ilya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#создаем счетчик 
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

#создаем матрицу из корпуса
X_train = count_tf_idf.fit_transform(X_train)
X_test = count_tf_idf.transform(X_test)

#проверяем размер матрицы
X_train.shape, X_test.shape

((111699, 142106), (47872, 142106))

In [12]:
def f1_cv(model, X, y, cv=3):
    #функция для рачета F1-меры на кросс-валидаци
    return cross_val_score(model, X, y, scoring='f1', cv=3)

Модель логистической регрессии построим с регулярицией L2 и L1.

In [13]:
#определяем модель c регуляризацие L2
lr_model = LogisticRegression(penalty='l2', random_state=42, solver='liblinear')

#считаем метрику
print('F1:', f1_cv(lr_model, X_train, y_train).mean().round(3))

F1: 0.676


Посмотрим как сработает логистическая регрессия с регуляризацией L1.

In [14]:
#определяем модель c регуляризацие L1
lr_model = LogisticRegression(penalty='l1', random_state=42, solver='liblinear', C=10)

#считаем метрику
print('F1:', f1_cv(lr_model, X_train, y_train).mean().round(3))

F1: 0.761


L1 регулягизация значительно лучше. 

In [15]:
#протестируем модель
lr_model.fit(X_train, y_train)

pred = lr_model.predict(X_test)

print('F1-мера на тестовой выборке:', f1_score(y_test, pred).round(3))

F1-мера на тестовой выборке: 0.773


Целевое значение метрики F1 получено. Далее протестируем модель Bert. Попробуем улучшить результат. 

In [16]:
#удалим выборки, чтобы не перегружать память при работе с бертом
del X_train, X_test, y_train, y_test

### Bert + logreg

Для моделирования будем использовать DistilBert (более легкая и быстрая версия Bert).

In [17]:
#загружаем модель DistilBert
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#определяем токенайзер и предобученную модель
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [18]:
def filter(row):
    #функция для фильтрации примеров с количеством токенов больше 512
    if len(row) < 512:
        return row

#фильтруем сэмпл
df_sample = df_toxic.copy()
df_sample['text'] = df_sample['text'].apply(filter)
df_sample.dropna(inplace=True)
df_sample.isna().sum()

text     0
toxic    0
dtype: int64

In [19]:
#сделае сбалансированную выборку
df_sample = df_sample.groupby('toxic', group_keys=False).apply(lambda x: x.sample(200)).reset_index(drop=True)

In [20]:
#проверяем баланс классов
df_sample['toxic'].value_counts()

0    200
1    200
Name: toxic, dtype: int64

In [21]:
#токенизируем текст
tokenized = df_sample['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [22]:
#найдем максимальную длинну вектора
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [23]:
#преобразуем данные в матрицу
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

#преобразуем маску
attention_mask = np.where(padded != 0, 1, 0)

#посмотрим размерность паддинга
padded.shape

(400, 113)

In [24]:
#задаем размер батча
batch_size = 50

#создаем список для эмбеддингов
embeddings = []

#добавляем эмбеддинги
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [25]:
#собираем эбеддинги в матрицу
X = np.concatenate(embeddings)
y = df_sample['toxic']

#разделим на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [26]:
#обучаем логистическую регрессию 
bert_lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)

print('F1:', f1_cv(bert_lr, X_train, y_train).mean().round(3))

F1: 0.808


Метрика с моделью Берт гораздо выше. Проверим на тестовой выборке. 

In [27]:
#тестируем модель
y_pred = bert_lr.predict(X_test)

print('F1-мера на тестовой выборке:', f1_score(y_test, y_pred).round(3))

F1-мера на тестовой выборке: 0.832


Модель Bert превзошла TF-IDF. 

# 3. Выводы

- Предобработали тексты: очистка с помощью регулярных выражений, стекинг английских слов. 
- Построили модель TF-IDF плюс логистическую регрессию. Данная комбинация представляет хороший бейзлайн. Модель быстро обучается и дает высокий результат. 
- Загрузили предобученную модель Bert. Преобразовали данные в эмбеддинги. Обучили логистическую регрессию на эмбеддингах. Не смотря на малое количество примеров целевого класса, получили высокое значение метрики *F1*: 0.83. 